In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

%pylab inline
from textgen import *

m = TextGen(
    img_size=299, 
    code_dim=512, 
    rnn_type='LSTM', 
    cnn_type='InceptionV3',
    pooling='avg',
)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def validate(model, val_record, ckpt, batch_size=64, capacity=8, n_threads=6, epoch_size=10000, 
             inds_to_words_json='/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/inds_to_words.json'):
    ''''''
    with tf.device('/cpu:0'):
        with tf.variable_scope('BatchReader'):
            print('creating batch reader')

            # coordinator for tf queues
            coord =  tf.train.Coordinator()

            # read and preprocess training records
            x, y, cls = model.read_tfrecord(
                val_record, 
                batch_size=batch_size, 
                capacity=capacity, 
                n_threads=n_threads,
                n_epochs=None)
            x = model.preproc_img(x)
            y, _ = model.preproc_caption(y, random=False)

        #with tf.variable_scope('StagingArea'):
        #    print('creating staging area')

        #    # create training queue on GPU
        #    get, size, put = model.stage_data(
        #        [x, y, cls], capacity=capacity)
        #    x, y, cls = get

        with tf.variable_scope('Metrics'):
            print('getting metrics')

            y_hat = model.sample_recursively(x, continuous=False)
            table = model.create_table(inds_to_words_json)

            y_inds, y_hat_inds = [tf.argmax(c, axis=-1) for c in (y, y_hat)]
            y_strings, y_hat_strings = [tf.gather(table, c) for c in (y_inds, y_hat_inds)]

            def bleu(data, pred):
                bs = data.shape[0]
                score = 0
                for i in range(bs):
                    y, y_hat = [[w for w in words if not w in ('SOS', 'EOS')] \
                                for words in data[i], pred[i]]
                    score += sentence_bleu([y], y_hat, weights=[1], 
                                           smoothing_function=SmoothingFunction().method3)
                score /= bs
                return np.array(score, dtype='float32')

            bleu_score = tf.py_func(bleu, [y_strings, y_hat_strings], tf.float32)

        with tf.Session(graph=model.graph) as sess:
            print('starting threads')

            tf.train.start_queue_runners(sess=sess, coord=coord)
            #stop, threads = model.start_threads(
            #    sess, put, n_stage_threads=n_threads)

            print('initializing variables')
            sess.run(tf.global_variables_initializer())

            print('loading weights from '+ckpt)
            model.load_weights(ckpt)

            #print('finalizing graph')
            #model.graph.finalize()
            
            classes = set([67, 14, 75, 53, 64, 86, 56, 50, 76, 37, 
                           89, 57, 70, 41, 33, 43, 40, 60, 88, 72,  
                            4, 54,  2, 79, 80,  3, 47, 20, 65, 82, 
                           32, 19, 17, 46, 27, 18,  9, 74, 49, 39])
            n = 0.
            in_ = 0.
            out = 0.
            try:
                for _ in tqdm.trange(epoch_size):
                    if not coord.should_stop():
                        return sess.run([cls, x], {backend.learning_phase(): False})
                        n += 1
                        if len(set(c) - classes) == 0:
                            in_ += 1.
                        else:
                            out += 1.
                    else:
                        raise IOError, 'queues closed'
                print(in_/out)
            # exit behaviour: request thread stop, then wait for 
            # them to recieve message before exiting session context
            except:
                coord.request_stop()
                time.sleep(0.2)
                raise

In [ ]:
c, x = validate(m, 
 '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/train_2shot.tfrecord',
 '/home/paperspace/training/20shot/lr4e-4_decay25k_bs64_reg1e-4_clip1.0/ckpt_update-0000130000.h5', 
 batch_size=1
)

In [ ]:
# read and preprocess training records
coord = tf.train.Coordinator()
x, y, cls = m.read_tfrecord(
    '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/val_1shot.tfrecord', 
    batch_size=1, 
    capacity=1, 
    n_threads=1,
    n_epochs=None)

with tf.Session() as sess:
    tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(5):
        print(sess.run(cls))